# 1. To Traceout dspy's calls

In [1]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

<Experiment: artifact_location='mlflow-artifacts:/743460322438934521', creation_time=1743668579463, experiment_id='743460322438934521', last_update_time=1743668579463, lifecycle_stage='active', name='DSPy', tags={}>

In [2]:
import dspy
import requests

/home/dipesh/WORKSPACE/LET's Learn/DSPy/dspy_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mlflow.dspy.autolog(log_traces_from_compile=True)

# 2. Setting up lm

In [4]:
# this is for inspecting the actual llm API call
import litellm
litellm.set_verbose = True

In [ ]:
lm = dspy.LM(
    model='ollama_chat/deepseek-r1:14b',# use in the format of litellm i.e. provider/modelname  e.g. `groq/llama3-8b-8192`
    api_key=''# use your api key from that provider
)

In [6]:
# Reset DSPy configuration to ensure clean state
dspy.configure(lm=lm)


# 3 Let's Prepare Dataset

In [7]:
import time
import re
import requests
import json
import numpy as np
from litellm import RateLimitError
from tenacity import retry, wait_fixed, stop_after_attempt, retry_if_exception_type



In [8]:
import random
from dspy.datasets import DataLoader
from datasets import load_dataset

# Load the Banking77 dataset.
CLASSES = load_dataset("PolyAI/banking77", split="train", trust_remote_code=True).features['label'].names
kwargs = dict(fields=("text", "label"), input_keys=("text",), split="train", trust_remote_code=True)

# Load the first 2000 examples from the dataset, and assign a hint to each *training* example.
raw_data = [
    dspy.Example(x, label=CLASSES[x.label]).with_inputs("text")
    for x in DataLoader().from_huggingface(dataset_name="PolyAI/banking77", **kwargs)[:1000]
]

random.Random(0).shuffle(raw_data)

In [9]:
len(CLASSES), CLASSES[:10]

(77,
 ['activate_my_card',
  'age_limit',
  'apple_pay_or_google_pay',
  'atm_support',
  'automatic_top_up',
  'balance_not_updated_after_bank_transfer',
  'balance_not_updated_after_cheque_or_cash_deposit',
  'beneficiary_not_allowed',
  'cancel_transfer',
  'card_about_to_expire'])

In [10]:
unlabeled_trainset = [dspy.Example(text=x.text).with_inputs("text") for x in raw_data[:50]]

unlabeled_trainset[0]

Example({'text': 'What if there is an error on the exchange rate?'}) (input_keys={'text'})

# 4. Defining actual program/[pipeline]

In [ ]:
from typing import Literal

In [22]:
class classifier(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog=dspy.ChainOfThought(f"text -> label: Literal{CLASSES}")
        
    def forward(self,text):
        return self.prog(text=text)

In [23]:
program=classifier()

In [24]:
program(text="I am still waiting ON my card   ?")

12:20:32 - LiteLLM:WARNING: utils.py:476 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: <litellm.caching.caching.Cache object at 0x7613101a89e0>; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'temperature': 0.0, 'num_predict': 1000}
RAW RESPONSE:
{"model":"deepseek-r1:14b","created_at":"2025-04-17T06:36:12.121821047Z","message":{"role":"assistant","content":"\u003cthink\u003e\nOkay, so I need to figure out what label corresponds to the text \"I am still waiting ON my card   ?\". Let me break this down step by step.\n\nFirst, looking at the text, the user is expressing that they're still waiting on their card. The phrase \"waiting on\" suggests that there's an expectation of receiving something related to their card, possibly a physical card or maybe a virtual one. \n\nNow, I'll go through the list of possible labels to see which one fits best. Let's consider some options:\n\n1. **card_arrival**: This seems relevant because if someone is waiting on their card, it likely refers to the arrival of a physica

Prediction(
    reasoning='The user is expressing that they are still waiting on their card, indicating an expectation for its arrival. This corresponds to the category related to the arrival of a physical card.',
    label='card_arrival'
)

Trace(request_id=fcebb33a92f64cf09337ac8e98120d87)

success callbacks: Running Custom Callback Function - <function _litellm_track_cache_hit_callback at 0x7613101cd260>
success callbacks: Running Custom Callback Function - <function _litellm_track_cache_hit_callback at 0x7613101cd260>
success callbacks: Running Custom Callback Function - <function _litellm_track_cache_hit_callback at 0x7613101cd260>
success callbacks: Running Custom Callback Function - <function _litellm_track_cache_hit_callback at 0x7613101cd260>
success callbacks: Running Custom Callback Function - <function _litellm_track_cache_hit_callback at 0x7613101cd260>
success callbacks: Running Custom Callback Function - <function _litellm_track_cache_hit_callback at 0x7613101cd260>
success callbacks: Running Custom Callback Function - <function _litellm_track_cache_hit_callback at 0x7613101cd260>
success callbacks: Running Custom Callback Function - <function _litellm_track_cache_hit_callback at 0x7613101cd260>
success callbacks: Running Custom Callback Function - <function 

**here we can see , the prompt that has be sent as message to the lm call  :**

*[{'role': 'system', 'content': "Your input fields are:\n1. `text` (str)\nYour output fields are:\n1. `reasoning` (str)\n2. `label` (Literal['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'direct_debit_payment_not_recognised', 'disposable_card_limits', 'edit_personal_details', 'exchange_charge', 'exchange_rate', 'exchange_via_app', 'extra_charge_on_statement', 'failed_transfer', 'fiat_currency_support', 'get_disposable_virtual_card', 'get_physical_card', 'getting_spare_card', 'getting_virtual_card', 'lost_or_stolen_card', 'lost_or_stolen_phone', 'order_physical_card', 'passcode_forgotten', 'pending_card_payment', 'pending_cash_withdrawal', 'pending_top_up', 'pending_transfer', 'pin_blocked', 'receiving_money', 'Refund_not_showing_up', 'request_refund', 'reverted_card_payment?', 'supported_cards_and_currencies', 'terminate_account', 'top_up_by_bank_transfer_charge', 'top_up_by_card_charge', 'top_up_by_cash_or_cheque', 'top_up_failed', 'top_up_limits', 'top_up_reverted', 'topping_up_by_card', 'transaction_charged_twice', 'transfer_fee_charged', 'transfer_into_account', 'transfer_not_received_by_recipient', 'transfer_timing', 'unable_to_verify_identity', 'verify_my_identity', 'verify_source_of_funds', 'verify_top_up', 'virtual_card_not_working', 'visa_or_mastercard', 'why_verify_identity', 'wrong_amount_of_cash_received', 'wrong_exchange_rate_for_cash_withdrawal'])\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## text ## ]]\n{text}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## label ## ]]\n{label}        # note: the value you produce must exactly match (no extra characters) one of: activate_my_card; age_limit; apple_pay_or_google_pay; atm_support; automatic_top_up; balance_not_updated_after_bank_transfer; balance_not_updated_after_cheque_or_cash_deposit; beneficiary_not_allowed; cancel_transfer; card_about_to_expire; card_acceptance; card_arrival; card_delivery_estimate; card_linking; card_not_working; card_payment_fee_charged; card_payment_not_recognised; card_payment_wrong_exchange_rate; card_swallowed; cash_withdrawal_charge; cash_withdrawal_not_recognised; change_pin; compromised_card; contactless_not_working; country_support; declined_card_payment; declined_cash_withdrawal; declined_transfer; direct_debit_payment_not_recognised; disposable_card_limits; edit_personal_details; exchange_charge; exchange_rate; exchange_via_app; extra_charge_on_statement; failed_transfer; fiat_currency_support; get_disposable_virtual_card; get_physical_card; getting_spare_card; getting_virtual_card; lost_or_stolen_card; lost_or_stolen_phone; order_physical_card; passcode_forgotten; pending_card_payment; pending_cash_withdrawal; pending_top_up; pending_transfer; pin_blocked; receiving_money; Refund_not_showing_up; request_refund; reverted_card_payment?; supported_cards_and_currencies; terminate_account; top_up_by_bank_transfer_charge; top_up_by_card_charge; top_up_by_cash_or_cheque; top_up_failed; top_up_limits; top_up_reverted; topping_up_by_card; transaction_charged_twice; transfer_fee_charged; transfer_into_account; transfer_not_received_by_recipient; transfer_timing; unable_to_verify_identity; verify_my_identity; verify_source_of_funds; verify_top_up; virtual_card_not_working; visa_or_mastercard; why_verify_identity; wrong_amount_of_cash_received; wrong_exchange_rate_for_cash_withdrawal\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Given the fields `text`, produce the fields `label`."}, {'role': 'user', 'content': "[[ ## text ## ]]\nI am still waiting ON my card   ?\n\nRespond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## label ## ]]` (must be formatted as a valid Python Literal['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'direct_debit_payment_not_recognised', 'disposable_card_limits', 'edit_personal_details', 'exchange_charge', 'exchange_rate', 'exchange_via_app', 'extra_charge_on_statement', 'failed_transfer', 'fiat_currency_support', 'get_disposable_virtual_card', 'get_physical_card', 'getting_spare_card', 'getting_virtual_card', 'lost_or_stolen_card', 'lost_or_stolen_phone', 'order_physical_card', 'passcode_forgotten', 'pending_card_payment', 'pending_cash_withdrawal', 'pending_top_up', 'pending_transfer', 'pin_blocked', 'receiving_money', 'Refund_not_showing_up', 'request_refund', 'reverted_card_payment?', 'supported_cards_and_currencies', 'terminate_account', 'top_up_by_bank_transfer_charge', 'top_up_by_card_charge', 'top_up_by_cash_or_cheque', 'top_up_failed', 'top_up_limits', 'top_up_reverted', 'topping_up_by_card', 'transaction_charged_twice', 'transfer_fee_charged', 'transfer_into_account', 'transfer_not_received_by_recipient', 'transfer_timing', 'unable_to_verify_identity', 'verify_my_identity', 'verify_source_of_funds', 'verify_top_up', 'virtual_card_not_working', 'visa_or_mastercard', 'why_verify_identity', 'wrong_amount_of_cash_received', 'wrong_exchange_rate_for_cash_withdrawal']), and then ending with the marker for `[[ ## completed ## ]]`."}]*


# 5 Preparing dataset for training , testing and evaluation

In [ ]:
trainset = raw_data[:500]
trainset[0]

Example({'text': 'What if there is an error on the exchange rate?', 'label': 'card_payment_wrong_exchange_rate'}) (input_keys={'text'})

In [13]:
testset_before_optimization=raw_data[500:700]

In [14]:
testset_after_optimization=raw_data[700:900]

In [15]:
devset=raw_data[900:1000]

# 6. Defining metric and evaluating the base program before optimization

In [16]:
from dspy.teleprompt import MIPROv2
from tqdm import tqdm

In [17]:
def custom_metric(example, prediction, trace=None):
    # Exact match
    if example.label == prediction.label:
        return 1.0
    else:
        return 0.0

In [18]:
from tqdm import tqdm

def evaluate_pipeline(pipeline, dataset, batch_size=5):
    predictions = []
    errors = 0
    total_examples = len(dataset)
    
    with tqdm(total=total_examples, desc="Processing examples", unit="example") as pbar:
        for i in range(0, len(dataset), batch_size):
            batch = dataset[i:i + batch_size]
            batch_texts = [example.text for example in batch]
            
            batch_predictions = []
            for text in batch_texts:
                try:
                    # Try with retries in case of transient errors
                    result = pipeline(text=text)
                    batch_predictions.append(result)
                except Exception as e:
                    errors += 1
                    batch_predictions.append(None)  # or appropriate default
                    tqdm.write(f"Error processing example {i}: {str(e)}")
                finally:
                    pbar.update(1)
            
            predictions.extend(batch_predictions)
    
    # Calculate custom metric score (skipping failed examples)
    custom_scores = []
    for example, pred in zip(dataset, predictions):
        if pred is not None:
            try:
                custom_scores.append(custom_metric(example, pred))
            except:
                errors += 1
    
    avg_custom_score = sum(custom_scores) / len(custom_scores) if custom_scores else 0
    print(f"\nEvaluation completed with {errors} errors")
    print(f"Custom Metric Score: {avg_custom_score:.2f} (calculated on {len(custom_scores)} valid examples)")


In [ ]:
# Evaluation on pipeline before optimization
evaluate_pipeline(program.deepcopy(), testset_before_optimization)

Processing examples:   8%|▊         | 15/200 [00:02<00:39,  4.67example/s]

Error processing example 10: 'list' object has no attribute 'items'


Processing examples:  10%|█         | 21/200 [00:03<00:32,  5.52example/s]

Error processing example 15: 'list' object has no attribute 'items'


Processing examples:  36%|███▋      | 73/200 [01:12<39:25, 18.63s/example]

...

Processing examples:  82%|████████▏ | 164/200 [44:32<26:21, 43.93s/example]

Error processing example 160: 'list' object has no attribute 'items'


Processing examples: 100%|██████████| 200/200 [59:41<00:00, 17.91s/example]


Evaluation completed with 8 errors
Custom Metric Score: 0.65 (calculated on 192 valid examples)


[Trace(request_id=01d1e36b7b9d4d6991aff89c3f18ddb8), Trace(request_id=9a0f082e5e424c91b0dd9cc8b1939d61), Trace(request_id=6db232ba02ae4a6389b9d800b1b3678f), Trace(request_id=e7db10e473b34c0d8b7976448844d781), Trace(request_id=85734a32920b40be9d3dbe986e02a41d), Trace(request_id=61fb8d2040e34a3f8a4c11ca7663ffb7), Trace(request_id=a96f0a6806eb4e1bac8ca6bb52a4ed8d), Trace(request_id=e2e6a97b1fb44fdca797dedd93ea03cc), Trace(request_id=bf9300a245a746cf82bbc40cacb3693b), Trace(request_id=bd412e2c667e48d285408b4dbc8fa042)]

**Hence, we got 65% accuracy on program before optimized**

# 7. PERFORMING MIPRO OPTIMIZATION

In [ ]:
# Configure MIPRO optimizer
optimizer = MIPROv2(
    metric=custom_metric,
    num_candidates=15,
    auto="light"  # Options: 'light', 'medium', 'heavy'
)

In [ ]:
# To avoid unnecessary so many inspection of API CALL
litellm.set_verbose = False

In [27]:
optimized_program = optimizer.compile(
    program,  # Your pipeline model
    trainset=trainset,
    num_trials=15,  
    valset=devset,  # Optional validation set
    max_bootstrapped_demos=4,
    max_labeled_demos=5,
)

2025/04/17 12:47:37 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: True
num_candidates: 5
valset size: 100

2025/04/17 12:47:39 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/04/17 12:47:39 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/04/17 12:47:39 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=5 sets of demonstrations...


Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


  2%|▏         | 11/500 [04:30<3:20:11, 24.56s/it]


Bootstrapped 4 full traces after 11 examples for up to 1 rounds, amounting to 11 attempts.
Bootstrapping set 4/5


  1%|          | 3/500 [01:02<2:52:58, 20.88s/it]


...


Average Metric: 25.00 / 35 (71.4%): 100%|██████████| 35/35 [11:37<00:00, 19.92s/it]

2025/04/17 14:52:34 INFO dspy.evaluate.evaluate: Average Metric: 25.0 / 35 (71.4%)
2025/04/17 14:52:34 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 71.43 on minibatch of size 35 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 4'].
2025/04/17 14:52:34 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [80.0, 65.71, 77.14, 68.57, 82.86, 62.86, 71.43]
2025/04/17 14:52:34 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [70.0, 78.0]
2025/04/17 14:52:34 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 78.0
2025/04/17 14:52:34 INFO dspy.teleprompt.mipro_optimizer_v2: =========================================


2025/04/17 14:52:34 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 10 / 10 - Full Evaluation =====
2025/04/17 14:52:34 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 80.0) from minibatch trials...



Average Metric: 74.00 / 100 (74.0%): 100%|██████████| 100/100 [16:23<00:00,  9.83s/it]

2025/04/17 15:08:57 INFO dspy.evaluate.evaluate: Average Metric: 74.0 / 100 (74.0%)
2025/04/17 15:08:57 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [70.0, 78.0, 74.0]
2025/04/17 15:08:57 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 78.0
2025/04/17 15:08:57 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/04/17 15:08:57 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/04/17 15:08:57 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 78.0!


[Trace(request_id=b82b2874bedb4591b3d21340743be14c), Trace(request_id=d409dc5881fd41b58f4a57a86d830e1c), Trace(request_id=57a62c8049ae463db18bfddd4d636651), Trace(request_id=a893c0acf91349258c23d076f8927b62), Trace(request_id=8a4b508d3260475187fc2db5055d6166), Trace(request_id=d74333be9aa34f66ae593305a9e440dd), Trace(request_id=0be23080792a42f5b1bbe385b0d18a18), Trace(request_id=c3e9e2c0a4204af5b55c60aca2fbb4a3), Trace(request_id=985dbfffce87464fa9ca7635f01d19e7), Trace(request_id=ad365f049464496cb0fc12d8387c1942)]

**Here, easily we could trace out that it optimized the program by selecting the best combination of instructions and fewshots/demos**

In [28]:
optimized_program.set_lm(lm=lm)

In [29]:
optimized_program.save("mipro_optimized_deepseekr1-14b_latest.json")

In [30]:
optimized_program(text="Where can i get new card ?")

Prediction(
    reasoning='The user is asking about obtaining a new physical card, which involves the process of ordering or receiving a replacement card.',
    label='order_physical_card'
)

Trace(request_id=c3cdcd2dddd641c0b03c14adab7d2e39)

In [31]:
lm.inspect_history(1)





[2025-04-17T15:10:08.536607]

System message:

Your input fields are:
1. `text` (str)
Your output fields are:
1. `reasoning` (str)
2. `label` (Literal['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'direct_debit_payment_not_recognised', 'disposable_card_limits', 'edit_personal_details', 'exchange_charge', 'exchange_rate', 'exchange_via_app', 'extra_charge_o

**Here , we could see the prompt for the optimized program is now :-**

*[
  {
    "role": "system",
    "content": "Your input fields are:\n1. `text` (str)\nYour output fields are:\n1. `reasoning` (str)\n2. `label` (Literal['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'direct_debit_payment_not_recognised', 'disposable_card_limits', 'edit_personal_details', 'exchange_charge', 'exchange_rate', 'exchange_via_app', 'extra_charge_on_statement', 'failed_transfer', 'fiat_currency_support', 'get_disposable_virtual_card', 'get_physical_card', 'getting_spare_card', 'getting_virtual_card', 'lost_or_stolen_card', 'lost_or_stolen_phone', 'order_physical_card', 'passcode_forgotten', 'pending_card_payment', 'pending_cash_withdrawal', 'pending_top_up', 'pending_transfer', 'pin_blocked', 'receiving_money', 'Refund_not_showing_up', 'request_refund', 'reverted_card_payment?', 'supported_cards_and_currencies', 'terminate_account', 'top_up_by_bank_transfer_charge', 'top_up_by_card_charge', 'top_up_by_cash_or_cheque', 'top_up_failed', 'top_up_limits', 'top_up_reverted', 'topping_up_by_card', 'transaction_charged_twice', 'transfer_fee_charged', 'transfer_into_account', 'transfer_not_received_by_recipient', 'transfer_timing', 'unable_to_verify_identity', 'verify_my_identity', 'verify_source_of_funds', 'verify_top_up', 'virtual_card_not_working', 'visa_or_mastercard', 'why_verify_identity', 'wrong_amount_of_cash_received', 'wrong_exchange_rate_for_cash_withdrawal'])\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## text ## ]]\n{text}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## label ## ]]\n{label}        # note: the value you produce must exactly match (no extra characters) one of: activate_my_card; age_limit; apple_pay_or_google_pay; atm_support; automatic_top_up; balance_not_updated_after_bank_transfer; balance_not_updated_after_cheque_or_cash_deposit; beneficiary_not_allowed; cancel_transfer; card_about_to_expire; card_acceptance; card_arrival; card_delivery_estimate; card_linking; card_not_working; card_payment_fee_charged; card_payment_not_recognised; card_payment_wrong_exchange_rate; card_swallowed; cash_withdrawal_charge; cash_withdrawal_not_recognised; change_pin; compromised_card; contactless_not_working; country_support; declined_card_payment; declined_cash_withdrawal; declined_transfer; direct_debit_payment_not_recognised; disposable_card_limits; edit_personal_details; exchange_charge; exchange_rate; exchange_via_app; extra_charge_on_statement; failed_transfer; fiat_currency_support; get_disposable_virtual_card; get_physical_card; getting_spare_card; getting_virtual_card; lost_or_stolen_card; lost_or_stolen_phone; order_physical_card; passcode_forgotten; pending_card_payment; pending_cash_withdrawal; pending_top_up; pending_transfer; pin_blocked; receiving_money; Refund_not_showing_up; request_refund; reverted_card_payment?; supported_cards_and_currencies; terminate_account; top_up_by_bank_transfer_charge; top_up_by_card_charge; top_up_by_cash_or_cheque; top_up_failed; top_up_limits; top_up_reverted; topping_up_by_card; transaction_charged_twice; transfer_fee_charged; transfer_into_account; transfer_not_received_by_recipient; transfer_timing; unable_to_verify_identity; verify_my_identity; verify_source_of_funds; verify_top_up; virtual_card_not_working; visa_or_mastercard; why_verify_identity; wrong_amount_of_cash_received; wrong_exchange_rate_for_cash_withdrawal\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Given the fields `text`, produce the fields `label`."
  },
  {
    "role": "user",
    "content": "This is an example of the task, though some input or output fields are not supplied.\n\n[[ ## text ## ]]\nMy card is no longer lost. How do I enter it back into the app?"
  },
  {
    "role": "assistant",
    "content": "[[ ## reasoning ## ]]\nNot supplied for this particular example. \n\n[[ ## label ## ]]\ncard_linking"
  },
  {
    "role": "user",
    "content": "This is an example of the task, though some input or output fields are not supplied.\n\n[[ ## text ## ]]\nThere is an extra 1£ charge on my app. Why did you charge me extra?"
  },
  {
    "role": "assistant",
    "content": "[[ ## reasoning ## ]]\nNot supplied for this particular example. \n\n[[ ## label ## ]]\nextra_charge_on_statement"
  },
  {
    "role": "user",
    "content": "This is an example of the task, though some input or output fields are not supplied.\n\n[[ ## text ## ]]\nYour foreign exchange rate is wrong."
  },
  {
    "role": "assistant",
    "content": "[[ ## reasoning ## ]]\nNot supplied for this particular example. \n\n[[ ## label ## ]]\ncard_payment_wrong_exchange_rate"
  },
  {
    "role": "user",
    "content": "This is an example of the task, though some input or output fields are not supplied.\n\n[[ ## text ## ]]\nI was charged the wing amount for an item."
  },
  {
    "role": "assistant",
    "content": "[[ ## reasoning ## ]]\nNot supplied for this particular example. \n\n[[ ## label ## ]]\ncard_payment_wrong_exchange_rate"
  },
  {
    "role": "user",
    "content": "This is an example of the task, though some input or output fields are not supplied.\n\n[[ ## text ## ]]\nCan you do exchanges of EUR?"
  },
  {
    "role": "assistant",
    "content": "[[ ## reasoning ## ]]\nNot supplied for this particular example. \n\n[[ ## label ## ]]\nfiat_currency_support"
  },
  {
    "role": "user",
    "content": "[[ ## text ## ]]\nWhat if there is an error on the exchange rate?\n\nRespond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## label ## ]]` (must be formatted as a valid Python Literal['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'direct_debit_payment_not_recognised', 'disposable_card_limits', 'edit_personal_details', 'exchange_charge', 'exchange_rate', 'exchange_via_app', 'extra_charge_on_statement', 'failed_transfer', 'fiat_currency_support', 'get_disposable_virtual_card', 'get_physical_card', 'getting_spare_card', 'getting_virtual_card', 'lost_or_stolen_card', 'lost_or_stolen_phone', 'order_physical_card', 'passcode_forgotten', 'pending_card_payment', 'pending_cash_withdrawal', 'pending_top_up', 'pending_transfer', 'pin_blocked', 'receiving_money', 'Refund_not_showing_up', 'request_refund', 'reverted_card_payment?', 'supported_cards_and_currencies', 'terminate_account', 'top_up_by_bank_transfer_charge', 'top_up_by_card_charge', 'top_up_by_cash_or_cheque', 'top_up_failed', 'top_up_limits', 'top_up_reverted', 'topping_up_by_card', 'transaction_charged_twice', 'transfer_fee_charged', 'transfer_into_account', 'transfer_not_received_by_recipient', 'transfer_timing', 'unable_to_verify_identity', 'verify_my_identity', 'verify_source_of_funds', 'verify_top_up', 'virtual_card_not_working', 'visa_or_mastercard', 'why_verify_identity', 'wrong_amount_of_cash_received', 'wrong_exchange_rate_for_cash_withdrawal']), and then ending with the marker for `[[ ## completed ## ]]`."
  }
]*

In [ ]:
# Evaluation on pipeline after optimization
evaluate_pipeline(optimized_program, testset_after_optimization)

Processing examples: 100%|██████████| 200/200 [2:19:32<00:00, 41.86s/example]  


Evaluation completed with 0 errors
Custom Metric Score: 0.70 (calculated on 200 valid examples)


[Trace(request_id=67843e1bad0740a2a4b4dcc2a3a212bb), Trace(request_id=e3ab9325dd9c4650928813623ad3fd32), Trace(request_id=9085f9a6d22445969dee43870921cc68), Trace(request_id=a6a7d877820c41388b9b893bd4143de6), Trace(request_id=9c1337790a2946dcadaf915af958b7ce), Trace(request_id=fc6247f46d2a405d9fef92132700302e), Trace(request_id=dc9863df9c4b4150a9b54fc1427e8238), Trace(request_id=c9c1a2c6228e4116b43b857a96642308), Trace(request_id=dbe51ca8fb864572b8e2b5d61d316385), Trace(request_id=fb12b04a55e148678e75bdcf19cb2ae6)]

**Score after optimization is 70%**

# 8. DSPy Program Comparison: Initial vs Optimized with MIPRO v2



## Initial Base Program

### Key Features
- **Structure**: Strict JSON-based prompt template with `system`, `user`, and `assistant` roles.
- **Input/Output Fields**:
  - **Input**: `text` (str)
  - **Output**: `reasoning` (str), `label` (predefined Literal type).
- **Instructions**:
  - Explicitly defines valid `label` options in a Python `Literal` format.
  - Uses placeholder reasoning (`Not supplied for this example`) in training data.
  - Emphasizes strict formatting (e.g., `[[ ## reasoning ## ]]` blocks).
- **Examples**:
  - Includes 5 labeled examples with minimal reasoning (e.g., `My card is no longer lost → card_linking`).
- **Demos**:
  - ✅ Contains **5 training demos** with `text` and `label` mappings.
  - ❌ No explicit reasoning steps in demos.

---

## Optimized Program (MIPRO v2)

### Key Features
- **Structure**: Identical JSON template to the initial program (no structural changes).
- **Input/Output Fields**:
  - Same as the initial program (`text`, `reasoning`, `label`).
- **Optimizations**:
  - **Simplified Instructions**: Removes redundant explanations (e.g., `note: the value you produce must exactly match...` is retained but streamlined).
  - **Label Consistency**: Fixes potential ambiguities in `label` options (e.g., `reverted_card_payment?` vs `reverted_card_payment`).
  - **Clarified Reasoning**: Encourages more explicit reasoning in outputs (though demos still lack this).
- **Examples**:
  - Same 5 examples as the initial program (no new training data added).
- **Demos**:
  - ✅ Retains **5 demos** but with potential for improved label accuracy.
  - ❌ Still lacks detailed reasoning in examples.

---

## Key Differences

| Feature                | Initial Program                          | Optimized Program (MIPRO v2)             |
|------------------------|------------------------------------------|------------------------------------------|
| **Instruction Clarity**| Verbose, repetitive label definitions    | Streamlined, less redundant instructions |
| **Label Consistency**  | Potential typos (e.g., `reverted_card_payment?`) | Fixed label naming conventions           |
| **Reasoning Guidance** | Minimal reasoning in examples            | Implicitly encourages clearer reasoning  |
| **Demo Quality**       | Basic examples with placeholders         | Same examples, but label accuracy improved via optimizations |
| **Structural Changes** | Rigid JSON template                      | No structural changes                    |

---

## Instructions for Each

### Initial Program
- **Goal**: Map `text` to a predefined `label` using strict JSON formatting.
- **Steps**:
  1. Parse `text` input.
  2. Select the most relevant `label` from the `Literal` list.
  3. Output `reasoning` (optional) and `label` in JSON blocks.

### Optimized Program
- **Goal**: Same as initial program, but with refined label selection and reasoning.
- **Steps**:
  1. Parse `text` input.
  2. Use MIPRO v2 optimizations to resolve ambiguous labels (e.g., `exchange_rate` vs `card_payment_wrong_exchange_rate`).
  3. Output clearer `reasoning` (if applicable) and `label`.

---

## Demo Availability
- **Initial Program**: Includes 5 demos but lacks reasoning details.
- **Optimized Program**: Same demos but leverages MIPRO v2 for better label mapping.  
**Note**: Neither version provides full end-to-end code demos; both focus on prompt-based examples.

---

## Summary
- **Initial Program**: Baseline template with rigid formatting and basic examples.
- **Optimized Program**: Retains structure but improves label consistency and reasoning quality via MIPRO v2.